# About: Moodleコンテナの起動

---

Moodleコンテナの起動と初期設定を行います

## 構成

![Moodleコンテナの起動](image/moodle-3-01.png)

## グループ名の指定

このNotebookの操作対象となるAnsibleのグループ名を設定します。

> グループ名を指定することにより、他のNotebookで設定した値を引き継ぐことが出来ます。

In [1]:
target_group = 'Moodle'

# 前提条件の確認

事前にNotebook「[02_VCノードの起動](02_VCノードの起動.ipynb)」が実行されていて、既にVCノードが起動していることなどを確認します。

パラメータを記録したファイルが存在することを確認します。

In [2]:
import os
if not os.path.exists(os.path.join('group_vars', target_group + '.yml')):
    raise RuntimeError("ERROR: not exists {}".format(target_group + '.yml'))

VCノードが起動されており、Notebook環境から Ansible で接続できることを確認します。

In [3]:
!ansible -m ping {target_group}

172.30.2.100 | SUCCESS => {
    "changed": false, 
    "failed": false, 
    "ping": "pong"
}


# パラメータの設定

![パラメータの設定](image/moodle-3-03.png)

## Moodle

### バージョン

インストールするMoodleのバージョンを指定します。

ここで指定する値はMoodleの[gitレポジトリ](https://github.com/moodle/moodle)の[タグ名](https://github.com/moodle/moodle/tags)と一致している必要があります。

In [4]:
# (例)
# moodle_version = 'v3.1.12'

moodle_version = 'v3.1.12'

#### チェックと保存

In [5]:
# タグ名の取得
import six
from six.moves.urllib.request import urlopen
import json
%run scripts/group.py

response = urlopen(
    'https://api.github.com/repos/moodle/moodle/git/refs/tags')
moodle_heads = [
    x['ref'].replace('refs/tags/', '')
    for x in json.loads(response.read().decode('utf-8'))]
response.close()

# チェック
if moodle_version not in moodle_heads:
    raise RuntimeError("ERROR: not exists: {}".format(moodle_version))

# 保存
update_group_vars(
    target_group,
    moodle_version=moodle_version,
)

### 管理者ユーザ

Moodleの管理者ユーザ名を指定します。

In [6]:
moodle_admin_name = 'admin'

#### 保存

In [7]:
update_group_vars(
    target_group,
    moodle_admin_name=moodle_admin_name,
)

### URL

`config.php`の`$CFG->wwwroot`に設定するURLを指定します。

ハンズオン環境では事前に設定しているVCノードのホスト名を元にURLを設定します。次のセルを実行するとURLの値を確認することが出来ます。

In [8]:
%run scripts/group.py

moodle_url = 'https://{}'.format(load_group_var(target_group, 'moodle_hostname'))

print(moodle_url)

https://moodle.example.org


#### 保存

In [9]:
if 'moodle_url' in vars():
    update_group_vars(
        target_group,
        moodle_url=moodle_url,
    )

### コンテナイメージ

Moodleのバージョンによって必要となるPHP のバージョンは異なります。またPHPのモジュールを追加したい場合等を考慮して、
アプリケーションテンプレートのNotebookではMoodleのコンテナイメージを事前に用意するのではなく設定されたパラメータに
応じたコンテナイメージを構築手順のなかでビルドすることを想定しています。

ハンズオン環境では設定されるパラメータなどが限定されていることと、コンテナイメージのビルドに必要となる時間を節約する
ために、事前にコンテナイメージを用意しました。ここでは事前に設定しているコンテナイメージ名の確認のみを行います。

In [10]:
%run scripts/group.py
moodle_image_name = load_group_var(target_group, 'moodle_image_name')
print(moodle_image_name)

192.168.2.1:5001/centos7-moodle


## MySQL

### コンテナイメージ

データベースコンテナのイメージ名を指定してください。指定可能なイメージは[MySQL](https://hub.docker.com/r/library/mysql/) または [MariaDB ](https://hub.docker.com/_/mariadb/) です。

In [11]:
# (例)
# database_image_name = 'mysql:8.0.11'
# database_image_name = 'mysql:5.7'
# database_image_name = 'mariadb:10.2'

database_image_name = 'mysql:5'

#### チェックと保存

設定された値をチェックして問題がなければ group_vars ファイルに保存します。以下のチェックを行っています。

* イメージ名が `mysql`, `mariadb` のいずれかであること
* Moodle のバージョンが 3.1, 3.2 の場合、MySQL 8 相当のイメージが指定されていないこと

> 現時点では Moodle 3.1, 3.2 で MySQL 8を使用するとインストールに失敗することを確認しているためその組み合わせが指定された場合はエラー
としてます(2018/4/25確認)。

In [12]:
import re
%run scripts/group.py
moodle_version = load_group_var(target_group, 'moodle_version')

# 設定値のチェック
w = database_image_name.split(':')
if w[0] not in {'mysql', 'mariadb'}:
    raise RuntimeError("ERROR: image name not supported: {}".format(w[0]))
if re.match(r'v3\.[12]', moodle_version) and (
        len(w) == 1 or w[1] == 'latest' or w[1].startswith('8')):
    raise RuntimeError(
        "ERROR: unsupported version: {}, {}".format(
            moodle_version, database_image_name))

# group_vars への保存
update_group_vars(
    target_group,
    database_image_name=database_image_name,
    database_type='mysqli' if w[0] == 'mysql' else 'mariadb',
)

### データベース名

Moodleが使用するデータベース名を指定してください。

In [13]:
mysql_database = 'moodle'

#### 保存

In [14]:
update_group_vars(
    target_group,
    mysql_database=mysql_database,
)

### 接続ユーザ

Moodleのデータベースに接続するためのデータベースのユーザ名を指定してください。

In [15]:
mysql_user_name = 'moodle'

#### 保存

In [16]:
update_group_vars(
    target_group,
    mysql_user_name=mysql_user_name,
)

## Nginx

### サーバ証明書

Nginxで使用するサーバ証明書が格納されているパスの設定を行います。

> ハンズオン環境では事前に設定をおこなっているのでここでは設定値を表示して確認のみ行います。

In [17]:
%run scripts/group.py

# 証明書のパス
proxy_tls_cert_path = load_group_var(target_group, 'proxy_tls_cert_path')
print(proxy_tls_cert_path)

# 鍵ファイルのパス
proxy_tls_key_path = load_group_var(target_group, 'proxy_tls_key_path')
print(proxy_tls_key_path)

/notebooks/notebook/handson/cert/server.crt
/notebooks/notebook/handson/cert/server.key


証明書の内容を確認します。

In [18]:
!openssl x509 -noout -text -in {proxy_tls_cert_path}

Certificate:
    Data:
        Version: 3 (0x2)
        Serial Number:
            03:5a:fd:ac:a5:5d:95:13:c4:35:ab:23:94:a6:55:44:64:71
    Signature Algorithm: sha256WithRSAEncryption
        Issuer: C=US, O=Let's Encrypt, CN=Let's Encrypt Authority X3
        Validity
            Not Before: May 21 06:55:30 2018 GMT
            Not After : Aug 19 06:55:30 2018 GMT
        Subject: CN=*.example.org
        Subject Public Key Info:
            Public Key Algorithm: rsaEncryption
                Public-Key: (2048 bit)
                Modulus:
                    00:d4:5d:5c:2b:97:18:0f:b3:f8:ab:97:2d:ab:16:
                    c9:35:8d:b2:83:d8:39:6c:cb:14:fd:e6:52:37:ca:
                    31:81:4d:f5:dd:45:df:3b:ec:e1:9a:17:0b:dd:3a:
                    af:49:d0:37:ee:59:e0:99:ae:53:2d:47:d2:b6:44:
                    1b:53:13:5e:df:8c:2e:60:b0:75:64:b2:52:a2:eb:
                    45:4b:f8:c7:ec:54:b3:bc:93:75:1a:22:42:83:4f:
                    33:e6:c3:5f:dd:74:98:96:6b:22:10:92

## 秘密情報の設定

構築するMoodle環境に関する秘密情報を Vault サーバに格納します。

対象となる秘密情報を以下に示します。

* MySQL のMoodleデータベースのパスワード
* Moodle の管理者パスワード

### 秘密情報の格納先の設定

秘密情報を格納する Vaultサーバ上のパスを設定します。

VCPの Vault サーバでは、2種類の格納場所を提供しています。

* secret/
  - 同一のVCコントローラを利用するユーザ間で共有している格納領域
* cubbyhole/
  - 利用者毎に異なる格納領域
  
通常は `cubbyhole` を指定します。

In [19]:
# (例)
# vault_moodle_path = 'cubbyhole/moodle'

vault_moodle_path = 'cubbyhole/moodle'

#### チェックと保存

既存のデータを上書きしないことを保証するために、指定された Vault サーバのパスにデータがないことを確認します。

> エラーになった場合は、格納先をパス(vault_moodle_path)を設定し直してください。

In [20]:
%run scripts/group.py
# ハンズオン環境では上書きチェックをスキップする
# vault_address = load_group_var(target_group, 'vault_address')
# vault_token = load_group_var(target_group, 'vcc_access_token')
# try:
#     !curl --silent --fail --header "X-Vault-Token:{vault_token}" \
#         "{vault_address}/v1/{vault_moodle_path}"
#     raise Exception("ERROR: {}: already exists".format(vault_moodle_path))
# except RuntimeError as e:
#     if not str(e).endswith(': 22'):
#         # エラーメッセージが"Unexpected exit code: 22" 以外の場合は想定していないエラーなので例外を再送
#        raise

# group_vars への保存
update_group_vars(
    target_group,
    vault_moodle_path=vault_moodle_path,
)

### 秘密情報の入力

MySQL のMoodleデータベースのパスワードを設定します。

次のセルを実行すると入力枠が表示されるのでパスワードを入力してください。

In [21]:
import getpass
mysql_user_password = getpass.getpass()

········


次にMoodle の管理者パスワードを設定します。

In [22]:
# moodle_admin_password = getpass.getpass()

# 本来は MySQLのパスワード同様にパスワード入力用のUIで入力しますが、
# 入力誤りを避けるためハンズオンでは文字列として設定します。
moodle_admin_password = 'Moodle0621'

### 保存

入力された秘密情報を Vault サーバに格納します。

In [23]:
import json
%run scripts/group.py

vault_address = load_group_var(target_group, 'vault_address')
vault_token = load_group_var(target_group, 'vcc_access_token')
vault_moodle_params = {
    'moodle_admin_password': moodle_admin_password,
    'mysql_user_password': mysql_user_password,
}

# Vaultサーバに秘密情報を格納する
!curl --fail --header "X-Vault-Token:{vault_token}" \
    --header "Content-Type: application/json" -X POST \
    --data '{json.dumps(vault_moodle_params)}' \
    "{vault_address}/v1/{vault_moodle_path}"

## チェック

設定項目に漏れがないかを確認します。

In [24]:
%run scripts/group.py
gvars = load_group_vars(target_group)
require_params = [
    'moodle_version', 'moodle_admin_name', 'moodle_url', 'moodle_image_name',
    'database_image_name', 'database_type', 'mysql_database', 'mysql_user_name',
    'proxy_tls_cert_path', 'proxy_tls_key_path', 'vault_moodle_path',
]

for x in require_params:
    if x not in gvars:
        raise RuntimeError("ERROR: not set {}".format(x))

## 保存したパラメータの確認

ここまで group_vars ファイルに保存したパラメータの内容を確認します。

In [25]:
!cat group_vars/{target_group}.yml

ansible_ssh_extra_args: -o StrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null
ansible_ssh_private_key_file: /home/bit_kun/.ssh/id_rsa_moodle
ansible_ssh_public_key_file: /home/bit_kun/.ssh/id_rsa_moodle.pub
ansible_user: ansible
container_version: 1
database_image_name: mysql:5
database_type: mysqli
docker_container_name_moodle: moodle-1
docker_container_name_mysql: mysql-1
moodle_admin_name: admin
moodle_dir_contents: /opt/moodle/data
moodle_dir_logs: /var/log/moodle
moodle_dir_scripts: /opt/moodle/php
moodle_shibboleth_enable: false
moodle_url: https://moodle.example.org
moodle_version: v3.1.12
mysql_database: moodle
mysql_dir_data: /opt/mysql
mysql_dir_logs: /var/log/mysql
mysql_user_name: moodle
project_dir: /home/ansible/moodle
project_template_name: moodle-https
proxy_config_dir: ''
proxy_config_name: nginx.conf
proxy_config_update_name: nginx-update.conf
proxy_dir_logs: /var/log/nginx
proxy_tls_enable: true
subvolume_name_moodle_contents: /opt/moodle/data/contents-1
subvol

# サーバの設定ファイル変更

![サーバの設定ファイル変更](image/moodle-3-04.png)

httpdサーバや MySQLの設定ファイルをコンテナに組み込まれているものから変更したい場合があります。ここではコンテナの設定ファイルを変更する手順を示します。対象とする設定ファイルを以下に示します。

* Moodleコンテナ
  - httpd.conf
  - php.ini
* MySQLコンテナ
  - my.cnf

## コンテナの設定ファイルを取得する

編集対象とする設定ファイルをコンテナから取得します。

はじめにコンテナから取得した設定ファイルを格納するディレクトリをNotebook環境に作成します。

In [26]:
from tempfile import mkdtemp 
import os
from datetime import datetime

parent = 'work_appc_config'
os.makedirs(parent, exist_ok=True)
work_dir = os.path.abspath(tempfile.mkdtemp(
    prefix=datetime.now().strftime('%Y%m%d_%H%M%S_'),
    dir=parent,
))
print(work_dir)

/notebooks/notebook/handson/moodle/work_appc_config/20180621_162236_adjvb4nw


後で参照するために、作成したディレクトリ名を保存します。

In [27]:
%run scripts/group.py
update_group_vars(
    target_group,
    appc_config_dir=os.path.abspath(work_dir),
)

VCノードの作業ディレクトリを作成します。

In [28]:
out = !ansible -a 'mktemp -d' {target_group}

bc_work_dir = out[1]
print(bc_work_dir)

/tmp/tmp.LgdEFF


Moodleコンテナ、MySQLコンテナを起動して設定ファイルを取得します。取得したファイルは先ほど作成したVCノードの作業ディレクトリ(`bc_work_dir`)に格納します。

In [29]:
# アプリケーションコンテナの起動(Moodle, MySQL)
!ansible -a 'docker run --rm -td --name moodle --entrypoint /bin/bash {{{{moodle_image_name}}}}' {target_group}
!ansible -a 'docker run --rm -td --name mysql {{{{database_image_name}}}} /bin/bash' {target_group}

# 設定ファイルのコピー
!ansible -a 'docker cp -L moodle:/etc/php.ini {bc_work_dir}' {target_group}
!ansible -a 'docker cp -L moodle:/etc/httpd/conf/httpd.conf {bc_work_dir}' {target_group}
!ansible -a 'docker cp -L mysql:/etc/mysql/my.cnf {bc_work_dir}' {target_group}

# アプリケーションコンテナの停止
!ansible -a 'docker stop moodle mysql' {target_group}

# 取得したファイルの確認
!ansible -a 'ls -l {bc_work_dir}' {target_group}

172.30.2.100 | SUCCESS | rc=0 >>
aa6f2fc161af441c783f2702013f5f50cd4ebe9f0b4494025e04b478809d1a6aUnable to find image '192.168.2.1:5001/centos7-moodle:latest' locally
latest: Pulling from centos7-moodle
697841bfe295: Pulling fs layer
5e86dc3c2abd: Pulling fs layer
fccb23a54c11: Pulling fs layer
c6777bffa04d: Pulling fs layer
56da11c7a800: Pulling fs layer
cbe2cccd828e: Pulling fs layer
b90f8782896c: Pulling fs layer
5a04bafacd15: Pulling fs layer
9636ff3e3b80: Pulling fs layer
4191968a11ac: Pulling fs layer
a6716d211456: Pulling fs layer
c6777bffa04d: Waiting
56da11c7a800: Waiting
cbe2cccd828e: Waiting
b90f8782896c: Waiting
5a04bafacd15: Waiting
9636ff3e3b80: Waiting
4191968a11ac: Waiting
a6716d211456: Waiting
5e86dc3c2abd: Verifying Checksum
5e86dc3c2abd: Download complete
fccb23a54c11: Verifying Checksum
fccb23a54c11: Download complete
697841bfe295: Download complete
cbe2cccd828e: Verifying Checksum
cbe2cccd828e: Download complete
b90f8782896c: Verifying Checksum
b90f8782896c: Downlo

設定ファイルをNotebook環境にコピーします。

In [30]:
!ansible -m fetch -a 'src={bc_work_dir}/httpd.conf dest={work_dir}/ flat=yes' {target_group}
!ansible -m fetch -a 'src={bc_work_dir}/php.ini dest={work_dir}/ flat=yes' {target_group}
!ansible -m fetch -a 'src={bc_work_dir}/my.cnf dest={work_dir}/ flat=yes' {target_group}

!ls -l {work_dir}

172.30.2.100 | SUCCESS => {
    "changed": true, 
    "checksum": "fdb1090d44c1980958ec96d3e2066b9a73bfda32", 
    "dest": "/notebooks/notebook/handson/moodle/work_appc_config/20180621_162236_adjvb4nw/httpd.conf", 
    "failed": false, 
    "md5sum": "f5e7449c0f17bc856e86011cb5d152ba", 
    "remote_checksum": "fdb1090d44c1980958ec96d3e2066b9a73bfda32", 
    "remote_md5sum": null
}
172.30.2.100 | SUCCESS => {
    "changed": true, 
    "checksum": "0aaec0f69f01bd844a6df11ed4cb222c698605d3", 
    "dest": "/notebooks/notebook/handson/moodle/work_appc_config/20180621_162236_adjvb4nw/php.ini", 
    "failed": false, 
    "md5sum": "3f92f543f23b83665db1b462a5a14854", 
    "remote_checksum": "0aaec0f69f01bd844a6df11ed4cb222c698605d3", 
    "remote_md5sum": null
}
172.30.2.100 | SUCCESS => {
    "changed": true, 
    "checksum": "28acdb29c88eb75cb98bfeea4b26a611f4fac295", 
    "dest": "/notebooks/notebook/handson/moodle/work_appc_config/20180621_162236_adjvb4nw/my.cnf", 
    "failed": false, 
  

VCノードの作業ディレクトリを削除します。

In [31]:
!ansible -a 'rm -rf {bc_work_dir}' {target_group}

 [WARNING]: Consider using file module with state=absent rather than running rm

172.30.2.100 | SUCCESS | rc=0 >>




## Moodleコンテナの設定ファイル

### httpd.confの編集

編集前にバックアップを作成します。

In [32]:
!cp {work_dir}/httpd.conf  {work_dir}/httpd.conf.orig

次のセルを実行すると httpd.conf へのリンクが表示されます。リンクをクリックすると httpd.conf を編集することができます。

In [33]:
%run scripts/nbutils.py
nb_html_link(os.path.abspath(os.path.join(work_dir, 'httpd.conf')), 'httpd.conf')

次のセルを実行すると編集前との差分を表示します。

In [34]:
try:
    !diff -u {os.path.join(work_dir, 'httpd.conf.orig')} \
        {os.path.join(work_dir, 'httpd.conf')}
except RuntimeError:
    pass

### php.iniの編集

編集前にバックアップを作成します。

In [35]:
!cp {work_dir}/php.ini {work_dir}/php.ini.orig

次のセルを実行すると php.ini へのリンクが表示されます。リンクをクリックすると php.ini を編集することができます。

In [36]:
%run scripts/nbutils.py
nb_html_link(os.path.abspath(os.path.join(work_dir, 'php.ini')), 'php.ini')

次のセルを実行すると編集前との差分を表示します。

In [37]:
try:
    !diff -u {os.path.join(work_dir, 'php.ini.orig')} \
        {os.path.join(work_dir, 'php.ini')}
except RuntimeError:
    pass

## MySQLコンテナの設定ファイル

### my.cnfの編集

編集前にバックアップを作成します。

In [38]:
!cp {work_dir}/my.cnf {work_dir}/my.cnf.orig

次のセルを実行すると my.cnf へのリンクが表示されます。リンクをクリックすると my.cnfを編集することができます。

In [39]:
%run scripts/nbutils.py
nb_html_link(os.path.abspath(os.path.join(work_dir, 'my.cnf')), 'my.cnf')

次のセルを実行すると編集前との差分を表示します。

In [40]:
try:
    !diff -u {os.path.join(work_dir, 'my.cnf.orig')} \
        {os.path.join(work_dir, 'my.cnf')}
except RuntimeError:
    pass

# データ領域の準備

![データ領域の準備](image/moodle-3-05.png)

## データベース用サブボリュームの作成

MySQLコンテナのデータを保管するためのサブボリューム(btrfs)を作成します。

既に同名のサブボリュームが存在していないことを確認します。次のセルを実行して `OK` と表示されたら同名のサブボリュームが存在していないことが確認できます。

In [56]:
target_group_database = load_group_var(target_group, 'target_group_database')
!ansible -m shell -b -a '\
    btrfs subvolume list {{{{subvolume_name_mysql}}}} 2> /dev/null \
    || echo OK' {target_group_database}

172.30.2.100 | SUCCESS | rc=0 >>
ID 257 gen 8 top level 5 path db-1



もし OK と表示されずサブボリュームが既に存在していた場合は、以下のコマンドを実行しサブボリュームを削除してください。

In [57]:
try:
    !ansible -b -a 'btrfs subvolume delete {{{{subvolume_name_mysql}}}}' {target_group_database}
except RuntimeError:
    pass

172.30.2.100 | SUCCESS | rc=0 >>
Delete subvolume (no-commit): '/opt/mysql/db-1'



サブボリュームを作成するパスにファイル等が存在しているとサブボリュームの作成がエラーとなるので、ファイルの削除を行います。

In [58]:
!ansible -b -a 'rm -rf {{{{subvolume_name_mysql}}}}' {target_group_database}

 [WARNING]: Consider using file module with state=absent rather than running rm

172.30.2.100 | SUCCESS | rc=0 >>




サブボリュームを作成します。

In [59]:
!ansible -b -a 'btrfs subvolume create {{{{subvolume_name_mysql}}}}' {target_group_database}

172.30.2.100 | SUCCESS | rc=0 >>
Create subvolume '/opt/mysql/db-1'



サブボリュームが作成できていることを確認します。

In [60]:
!ansible -b -a 'btrfs subvolume list {{{{subvolume_name_mysql}}}}' {target_group_database}

172.30.2.100 | SUCCESS | rc=0 >>
ID 258 gen 9 top level 5 path db-1



## moodledataデータ用サブボリュームの作成

Moodleコンテナがアップロードファイル、一時データ、セッションデータ等を保存するディレクトリ（moodledataデータディレクトリ）に対応するサブボリュームを作成します。

既に同名のサブボリュームが存在していないことを確認します。次のセルを実行して `OK` と表示されたら同名のサブボリュームが存在していないことが確認できます。

In [61]:
target_group_database = load_group_var(target_group, 'target_group_database')
try:
    !ansible -m shell -b -a '\
        btrfs subvolume list {{{{subvolume_name_moodle_contents}}}} 2> /dev/null \
        || echo OK' {target_group_database}
except RuntimeError:
    pass

172.30.2.100 | SUCCESS | rc=0 >>
ID 257 gen 10 top level 5 path data
ID 258 gen 11 top level 5 path php
ID 259 gen 10 top level 257 path data/contents-1
ID 260 gen 11 top level 258 path php/moodle-1



もし OK と表示されずサブボリュームが既に存在していた場合は、以下のコマンドを実行しサブボリュームを削除してください。

In [62]:
try:
    !ansible -ba 'btrfs subvolume delete {{{{subvolume_name_moodle_contents}}}}' {target_group_database}
except RuntimeError:
    pass

172.30.2.100 | SUCCESS | rc=0 >>
Delete subvolume (no-commit): '/opt/moodle/data/contents-1'



サブボリュームを作成するパスにファイル等が存在しているとサブボリュームの作成がエラーとなるので、ファイルの削除を行います。

In [63]:
!ansible -ba 'rm -rf {{{{subvolume_name_moodle_contents}}}}' {target_group_database}

 [WARNING]: Consider using file module with state=absent rather than running rm

172.30.2.100 | SUCCESS | rc=0 >>




サブボリュームを作成します。

In [64]:
!ansible -b -a 'btrfs subvolume create {{{{subvolume_name_moodle_contents}}}}' {target_group_database}

172.30.2.100 | SUCCESS | rc=0 >>
Create subvolume '/opt/moodle/data/contents-1'



サブボリュームが作成できていることを確認します。

In [65]:
!ansible -b -a 'btrfs subvolume list -o {{{{moodle_dir_contents}}}}' {target_group_database}

172.30.2.100 | SUCCESS | rc=0 >>
ID 261 gen 12 top level 257 path data/contents-1



## Moodle PHPファイル用サブボリュームの作成

MoodleコンテナがPHPファイルやプラグインを保存するためのサブボリュームを作成します。

既に同名のサブボリュームが存在していないことを確認します。次のセルを実行して `OK` と表示されたら同名のサブボリュームが存在していないことが確認できます。

In [66]:
target_group_database = load_group_var(target_group, 'target_group_database')
try:
    !ansible -m shell -b -a '\
        btrfs subvolume list {{{{subvolume_name_moodle_scripts}}}} 2> /dev/null \
        || echo OK' {target_group_database}
except RuntimeError:
    pass

172.30.2.100 | SUCCESS | rc=0 >>
ID 257 gen 12 top level 5 path data
ID 258 gen 11 top level 5 path php
ID 260 gen 11 top level 258 path php/moodle-1
ID 261 gen 12 top level 257 path data/contents-1



もし OK と表示されずサブボリュームが既に存在していた場合は、以下のコマンドを実行しサブボリュームを削除してください。

In [67]:
try:
    !ansible -b -a 'btrfs subvolume delete {{{{subvolume_name_moodle_scripts}}}}' {target_group_database}
except RuntimeError:
    pass

172.30.2.100 | SUCCESS | rc=0 >>
Delete subvolume (no-commit): '/opt/moodle/php/moodle-1'



サブボリュームを作成するパスにファイル等が存在しているとサブボリュームの作成がエラーとなるので、ファイルの削除を行います。

In [68]:
!ansible -b -a 'rm -rf {{{{subvolume_name_moodle_scripts}}}}' {target_group_database}

 [WARNING]: Consider using file module with state=absent rather than running rm

172.30.2.100 | SUCCESS | rc=0 >>




サブボリュームを作成します。

In [69]:
!ansible -b -a 'btrfs subvolume create {{{{subvolume_name_moodle_scripts}}}}' {target_group_database}

172.30.2.100 | SUCCESS | rc=0 >>
Create subvolume '/opt/moodle/php/moodle-1'



サブボリュームが作成できていることを確認します。

In [70]:
!ansible -b -a 'btrfs subvolume list -o {{{{moodle_dir_scripts}}}}' {target_group_database}

172.30.2.100 | SUCCESS | rc=0 >>
ID 262 gen 13 top level 258 path php/moodle-1



# アプリケーションコンテナの起動

![コンテナの起動](image/moodle-3-06.png)

## 設定ファイルの作成

アプリケーションコンテナを起動するのに必要となる設定ファイルを作成します。ここで作成する設定ファイルは以下の3つのファイルです。

* docker-compose.yml
* nginx.conf
* .env

まず作業ディレクトリを作成します。

In [71]:
%run scripts/docker_compose.py
comp_work_dir = make_compose_workdir()
comp_work_dir

'/notebooks/notebook/handson/moodle/compose/20180621_162657_zey1v94p'

`docker-compose.yml` と `nginx.conf` についてはテンプレートのファイルを用意してありますので、それを作業ディレクトリにコピーします。

In [72]:
!cp -rp {os.path.join( \
    'template/docker/compose', \
    load_group_var(target_group, 'project_template_name'), '*')} \
    {comp_work_dir}
!ls -Rl {comp_work_dir}

/notebooks/notebook/handson/moodle/compose/20180621_162657_zey1v94p:
total 8
-rw-r--r-- 1 bit_kun users 1931 Jun 20 15:34 docker-compose.yml
-rw-r--r-- 1 bit_kun users 1942 Jun 20 15:34 nginx.conf


docker-compose.yml はコンテナの構成や起動時のオプションが記述してある設定ファイルです。基本的にはテンプレートのファイルをそのまま利用できますが、個別に編集したい場合は次のセルを実行して表示されるリンクから編集を行ってください。

In [73]:
%run scripts/nbutils.py
nb_html_link(os.path.join(comp_work_dir, 'docker-compose.yml'), 'docker-compose.yml')

以下のセルを実行すると `docker-compose.yml` の編集前との差分を確認できます。

In [74]:
try:
    !diff -u {os.path.join( \
        'template/docker/compose', \
        load_group_var(target_group, 'project_template_name'), \
        'docker-compose.yml')} \
    {os.path.join(comp_work_dir, 'docker-compose.yml')}
except RuntimeError:
    pass

nginx.conf はリバースプロキシとして実行する Nginxコンテナの設定ファイルです。基本的にはテンプレートのファイルをそのまま利用できますが、個別に編集したい場合は次のセルを実行して表示されるリンクから編集を行ってください。

In [75]:
%run scripts/nbutils.py
nb_html_link(os.path.join(comp_work_dir, 'nginx.conf'), 'nginx.conf')

以下のセルを実行すると `nginx.conf` の編集前との差分を確認できます。

In [76]:
try:
    !diff -u {os.path.join( \
        'template/docker/compose', \
        load_group_var(target_group, 'project_template_name'), \
        'nginx.conf')} \
        {os.path.join(comp_work_dir, 'nginx.conf')} 
except RuntimeError:
    pass

`.env` はMySQLに関する環境変数を設定しているファイルです。このファイルは MySQLコンテナ、Moodleコンテナの両方から参照するので `docker-compose.yml` とは別のファイルとして書き出しています。

In [77]:
with open(os.path.join(comp_work_dir, '.env'), 'w') as f:
    f.write('''
MYSQL_DATABASE={{mysql_database}}
MYSQL_USER={{mysql_user_name}}
MYSQL_PASSWORD={{mysql_user_password}}
''')

## 設定ファイルの配置

作成した設定ファイルをVCノードに配置します。

設定ファイルの記述で `{{ }}` で記されている箇所はVCノードに配置する際に group_vars に記録されているパラメータの値で置換されます。これは Ansible の template の機能を利用して実現しています。ただし秘密情報については group_vars に記録されておらず Vault サーバのみに格納されているので、ここで値の取得を行います。

In [78]:
import json
import yaml
import tempfile
%run scripts/group.py

# Vaultサーバにアクセスするために必要となる情報を読み込む
vault_address = load_group_var(target_group, 'vault_address')
vault_token = load_group_var(target_group, 'vcc_access_token')

# Vaultサーバから秘密情報の取得
vault_moodle_path = load_group_var(target_group, 'vault_moodle_path')
out = !curl --silent --fail --header "X-Vault-Token:{vault_token}" \
    "{vault_address}/v1/{vault_moodle_path}"

# 秘密情報を一時ファイルに暗号化して保存する
moodle_params = dict([
    (k, v)
    for k, v in json.loads(''.join(out))['data'].items()
])
(th, secret_file) = tempfile.mkstemp('.yml', text=True)
with open(th, 'w+') as f:
    f.write(yaml.dump(moodle_params))

設定ファイルをVCノードに配置します。

In [79]:
target_group_rproxy = load_group_var(target_group, 'target_group_rproxy')
target_group_moodle = load_group_var(target_group, 'target_group_moodle')
target_group_database = load_group_var(target_group, 'target_group_database')

# 配置先のディレクトリを作成する 
!ansible -b -m file -a 'state=directory owner={{{{ansible_user}}}} \
    group={{{{ansible_user}}}} path={{{{project_dir}}}}' {target_group}
!ansible -m file -a 'state=directory path={{{{project_dir}}}}/ssl' {target_group}
!ansible -m file -a 'state=directory path={{{{project_dir}}}}/conf' {target_group}
!ansible -b -m file -a '\
    state=directory owner=999 group=999 \
    path={{{{mysql_dir_logs}}}}/{{{{docker_container_name_mysql}}}}'\
    {target_group_database}
!ansible -b -m file -a 'state=directory \
    path={{{{moodle_dir_logs}}}}/{{{{docker_container_name_moodle}}}}' \
    {target_group_moodle}
!ansible -b -m file -a 'state=directory path={{{{proxy_dir_logs}}}}' {target_group_rproxy}

# 設定ファイルの配置
for fname in ['docker-compose.yml', 'nginx.conf', '.env']:
    !ansible -m template -e "@{secret_file}" \
        -a 'src={os.path.join(comp_work_dir, fname)} \
        dest={{{{project_dir}}}}/{fname}' {target_group}
        
# 証明書ファイルの配置
!ansible -m copy -a 'src={{{{proxy_tls_cert_path}}}} \
    dest={{{{project_dir}}}}/ssl/server.crt' {target_group_rproxy}
!ansible -m copy -a 'src={{{{proxy_tls_key_path}}}} \
    dest={{{{project_dir}}}}/ssl/server.key' {target_group_rproxy}

# カスタマイズした設定ファイルの配置
for fname in ['php.ini', 'httpd.conf', 'my.cnf']:
    !ansible -m copy -a 'src={{{{appc_config_dir}}}}/{fname} \
        dest={{{{project_dir}}}}/conf/ backup=yes' {target_group}

172.30.2.100 | SUCCESS => {
    "changed": true, 
    "failed": false, 
    "gid": 1000, 
    "group": "ansible", 
    "mode": "0755", 
    "owner": "ansible", 
    "path": "/home/ansible/moodle", 
    "size": 4096, 
    "state": "directory", 
    "uid": 1000
}
172.30.2.100 | SUCCESS => {
    "changed": true, 
    "failed": false, 
    "gid": 1000, 
    "group": "ansible", 
    "mode": "0755", 
    "owner": "ansible", 
    "path": "/home/ansible/moodle/ssl", 
    "size": 4096, 
    "state": "directory", 
    "uid": 1000
}
172.30.2.100 | SUCCESS => {
    "changed": true, 
    "failed": false, 
    "gid": 1000, 
    "group": "ansible", 
    "mode": "0755", 
    "owner": "ansible", 
    "path": "/home/ansible/moodle/conf", 
    "size": 4096, 
    "state": "directory", 
    "uid": 1000
}
172.30.2.100 | SUCCESS => {
    "changed": true, 
    "failed": false, 
    "gid": 999, 
    "group": "ping", 
    "mode": "0755", 
    "owner": "999", 
    "path": "/var/log/mysql/mysql-1", 
    "size": 4

ローカル環境にある秘密情報を格納したファイルは不要になったので削除します。

In [80]:
import os
os.unlink(secret_file)

## コンテナの起動

docker-composeコマンドを実行してコンテナの起動を行います。

In [81]:
gvars = load_group_vars(target_group)
if 'moodle_nodes' in gvars:
    !ansible -a 'docker stack deploy -c {{{{project_dir}}}}/docker-compose.yml {target_group}' \
        {target_group_rproxy}
else:
    !ansible -m shell -a 'cd {{{{project_dir}}}} && docker-compose up -d' {target_group_rproxy}

172.30.2.100 | SUCCESS | rc=0 >>
latest: Pulling from library/nginx
Digest: sha256:3e2ffcf0edca2a4e9b24ca442d227baea7b7f0e33ad654ef1eb806fbd9bedcf0
Status: Downloaded newer image for nginx:latestCreating network "moodle_default" with the default driver
Pulling proxy (nginx:latest)...







正常に起動されたことを確認します。まれに起動に失敗していることがあるので、その場合は何度か再試行しています。

In [82]:
if 'moodle_nodes' not in gvars:
    %run scripts/utils.py
    
    def check_all_container_up():
        lines = !ansible -m shell -a 'cd {{{{project_dir}}}} && docker-compose ps' {target_group_rproxy}
        for cname in [gvars['docker_container_name_moodle'],
                gvars['docker_container_name_mysql'], 'proxy']:
            check_ansible_output_docker_compose(lines, cname, output=False)
        
    def do_docker_compose_up():
        !ansible -m shell -a 'cd {{{{project_dir}}}} && docker-compose up -d' {target_group_rproxy} 
    
    retry_exec(check_all_container_up, interval=3, retry_max=10, redo=do_docker_compose_up)

### Grafanaによる利用状況の確認

起動したアプリケーションコンテナのCPU, Memory 使用量などをGrafanaで確認してください。

In [83]:
from IPython.core.display import HTML
HTML(u'<a href="/grafana/" target="_blank">Grafana</a>')

### Moodleのインストール処理が完了するまで待つ

Moodleのインストール処理が完了してMoodleのサービスが開始されるまで数分程度を要します。ここではMoodleのサービスが開始されるまでの待ち合わせを行います。

> Moodleが起動するまでは `502 Bad Gateway` と表示されます。その間は、そのまま待ってください。

In [84]:
import time

# time.sleep(120)
moodle_url = 'https://{}'.format(
    load_group_var(target_group, 'vc_node_address'))

def check_not_bad_gateway():
    try:
        !curl -k -s -I {moodle_url} | grep 502
        raise Exception()
    except RuntimeError:
        pass

retry_exec(check_not_bad_gateway, err=Exception)

HTTP/1.1 502 Bad Gateway
HTTP/1.1 502 Bad Gateway
HTTP/1.1 502 Bad Gateway
HTTP/1.1 502 Bad Gateway
HTTP/1.1 502 Bad Gateway
HTTP/1.1 502 Bad Gateway
HTTP/1.1 502 Bad Gateway
HTTP/1.1 502 Bad Gateway
HTTP/1.1 502 Bad Gateway


# Moodle を利用できることを確認

起動したMoodleのアドレスにアクセスし、Moodle が利用できることを確認します。

## アドレスの確認

In [85]:
from IPython.core.display import HTML
HTML(u'<a href="{0}" target="_blank">{0}</a>'.format(load_group_var(target_group, 'moodle_url')))

## ライブログの確認

Moodle のライブログを確認します。次のセルを実行すると表示されるリンクをクリックしてください。

In [86]:
from IPython.core.display import HTML
HTML(u'<a href="{0}/report/loglive/index.php" target="_blank">{0}</a>'.format(
    load_group_var(target_group, 'moodle_url')))

# cronの設定

cron.phpを5分毎に実行するようVCノードの crontab を設定します。

In [87]:
target_group_moodle = load_group_var(target_group, 'target_group_moodle')
!ansible -m cron -a 'minute="*/5" job="docker exec {{{{docker_container_name_moodle}}}} \
    /usr/bin/php /var/www/html/admin/cli/cron.php > /dev/null" user="{{{{ansible_user}}}}" \
    name="{{{{docker_container_name_moodle}}}} cron.php"' {target_group_moodle}

172.30.2.100 | SUCCESS => {
    "changed": true, 
    "envs": [], 
    "failed": false, 
    "jobs": [
        "moodle-1 cron.php"
    ]
}


crontabに登録されたことを確認します。

In [88]:
!ansible -a 'crontab -l' {target_group_moodle}

172.30.2.100 | SUCCESS | rc=0 >>
#Ansible: moodle-1 cron.php
*/5 * * * * docker exec moodle-1     /usr/bin/php /var/www/html/admin/cli/cron.php > /dev/null

